In [ ]:
import os
import shutil
import pandas as pd
source_dir=r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\raw_data\archive_1\charaNet"
destination_dir=r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\raw_data_flat"
os.makedirs(destination_dir, exist_ok=True)
metadata=[]
for split in os.listdir(source_dir):
    split_path = os.path.join(source_dir, split)
    if os.path.isdir(split_path):
        for species in os.listdir(split_path):
            species_path = os.path.join(split_path, species)
            if os.path.isdir(species_path):
                for file in os.listdir(species_path):
                    if file.endswith(".wav") or file.endswith(".mp3") or file.endswith(".ogg"):
                        src_file = os.path.join(species_path, file)
                        new_filename = f"{species}_{split}_{file}"
                        dest_file = os.path.join(destination_dir, new_filename)
                        shutil.copy(src_file, dest_file)
                        metadata.append([new_filename, species, split])
df=pd.DataFrame(metadata, columns=["filename", "species", "split"])
df.to_csv(os.path.join(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection", "metadata.csv"), index=False)
print("Data flattened and metadata saved to metadata.csv")

Data flattened and metadata saved to metadata.csv


In [2]:
import os
import shutil
from pydub import AudioSegment

raw_flat = r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\raw_data_flat"
converted_folder = r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\processed_data"
denoised_folder = r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\processed_denoised"

os.makedirs(converted_folder, exist_ok=True)
os.makedirs(denoised_folder, exist_ok=True)

for file in os.listdir(raw_flat):
    if file.endswith((".mp3", ".ogg", ".flac", ".wav")):
        file_path = os.path.join(raw_flat, file)
        if not file.endswith(".wav"):
            audio = AudioSegment.from_file(file_path)
            new_filename = file.split('.')[0] + ".wav"
            dest_file = os.path.join(converted_folder, new_filename)
            audio.export(dest_file, format="wav")
        else:
            dest_file = os.path.join(converted_folder, file)
            shutil.copy(file_path, dest_file)

print("All files converted to wav and copied to processed_data folder")


All files converted to wav and copied to processed_data folder


In [9]:
import librosa
import noisereduce as nr
import soundfile as sf

def denoise_file(input_path,output_path):
    y, sr=librosa.load(input_path,sr=16000)
    y_denoised=nr.reduce_noise(y=y, sr=sr,prop_decrease=0.8)
    sf.write(output_path,y_denoised,sr)
    
for file in os.listdir(converted_folder):
    if file.endswith(".wav"):
        input_file=os.path.join(converted_folder,file)
        output_file=os.path.join(denoised_folder,file.split('.')[0]+"_denoised.wav")
        denoise_file(input_file,output_file)
print("denoising complete. Denoised files are in:",denoised_folder)

denoising complete. Denoised files are in: C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\processed_denoised


In [10]:
import os
import pandas as pd

converted_folder = r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\processed_data"
metadata_updated = []

for file in os.listdir(converted_folder):
    if file.endswith(".wav"):
        species = file.split("_")[0]
        split = file.split("_")[1] if len(file.split("_")) > 1 else "unknown"
        metadata_updated.append([file, species, split])

df_updated = pd.DataFrame(metadata_updated, columns=["filename", "species", "split"])
df_updated.to_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\metadata_updated.csv", index=False)
print("Metadata CSV updated!")


Metadata CSV updated!


In [11]:
import os
import librosa
import numpy as np
import pandas as pd

features = []
denoised_folder = r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\processed_denoised"

for file in os.listdir(denoised_folder):
    if file.endswith(".wav"):
        file_path = os.path.join(denoised_folder, file)
        y, sr = librosa.load(file_path, sr=16000)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        features.append([file] + mfcc_mean.tolist())

columns = ["filename"] + [f"mfcc_{i+1}" for i in range(13)]
df_features = pd.DataFrame(features, columns=columns)
df_features.to_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\\features.csv", index=False)
print("Feature extraction complete! Features saved to features.csv")


Feature extraction complete! Features saved to features.csv


In [7]:
import os
import librosa
import numpy as np
import pandas as pd

features = []
denoised_folder = r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\processed_denoised"

for file in os.listdir(denoised_folder):
    if file.endswith(".wav"):
        file_path = os.path.join(denoised_folder, file)
        y, sr = librosa.load(file_path, sr=16000)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        base_filename = file.replace("_denoised", "")
        features.append([base_filename] + mfcc_mean.tolist())

columns = ["filename"] + [f"mfcc_{i+1}" for i in range(13)]
df_features = pd.DataFrame(features, columns=columns)
df_features.to_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\features.csv", index=False)
print("Feature extraction complete! Features saved to features.csv")


Feature extraction complete! Features saved to features.csv


In [11]:
import pandas as pd

metadata = pd.read_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\metadata_updated.csv")
# Filter only training data
train_metadata = metadata[metadata['split'].str.lower() == "train"]
train_metadata.to_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\metadata_train.csv", index=False)
print("Training metadata CSV created!")


Training metadata CSV created!


In [14]:
import pandas as pd

metadata = pd.read_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\metadata_updated.csv")
unique_species = metadata['species'].unique()
print("Unique species in dataset:", unique_species)


Unique species in dataset: ['Asian Koel' 'Black Kite' 'Black-breasted Parrotbill'
 'Black-necked crane' 'Bristled Grassbird' 'Cheer Pheasant'
 'Common Cuckoo' 'Common Pochard' 'Common Wood Pigeon'
 'Eastern Imperial Eagle' 'Egyptian Vulture' 'Great Slaty Woodpecker'
 'Greater Spotted Eagle' 'Grey Treepie' 'Grey-crowned Prinia'
 'Grey-sided Thrush' 'Himalayan Monal' 'House Crow' 'House Sparrow'
 'Indian Spotted Eagle' 'Jerdon' 'Kashmir Flycatcher' 'Large-billed Crow'
 'Long-tailed Duck' 'Pallas' 'Red-billed Blue Magpie'
 'Rose-ringed Parakeet' 'Rufous Treepie' 'Rufous-necked Hornbill'
 'Rustic Bunting' 'Saker Falcon' 'Sarus Crane' 'Satyr Tragopan'
 'Slender-billed Babbler' 'Spiny Babbler' 'Spotted Dove' 'Steppe Eagle'
 'Swamp Francolin' 'Swamp Grass-babbler' 'White-throated Bushchat'
 'Wood Snipe']


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import joblib


metadata = pd.read_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\metadata_updated.csv")
features = pd.read_csv(r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\features.csv")

data = pd.merge(metadata, features, on="filename")
print("Merged training data shape:", data.shape)

non_endangered_species = ["housecrow", "housesparrow", "commonwoodpigeon"]

def standardize(name):
    return name.replace(" ", "").lower()


data["status"] = data["species"].apply(lambda x: "Non-Endangered" if standardize(x) in non_endangered_species else "Endangered")
data["status_numeric"] = data["status"].apply(lambda x: 0 if x.lower() == "non-endangered" else 1)


X = data[[f"mfcc_{i+1}" for i in range(13)]].values
y = data["status_numeric"].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)


y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))



joblib.dump(model, r"C:\Users\Varun\OneDrive\Desktop\Bird Sound and Endangered Detection\trained_model.pkl")
print("Model saved!")



Merged training data shape: (6542, 16)
              precision    recall  f1-score   support

           0       0.88      0.71      0.79       234
           1       0.94      0.98      0.96      1075

    accuracy                           0.93      1309
   macro avg       0.91      0.85      0.87      1309
weighted avg       0.93      0.93      0.93      1309

Model saved!
